<a href="https://colab.research.google.com/github/gamsingh/CS7643/blob/main/GCN%2BLSTM_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The model consists of two key modules:

*   **Meteorological Factor Spatio-Temporal Feature Extraction Module:**
This module constructs an undirected graph using Graph Convolutional Networks (GCN) for spatial encoding and Long Short-Term Memory (LSTM) networks for temporal encoding. The spatio-temporal features are then processed through an attention mechanism to integrate with PM2.5 data.

*   **PM2.5 Concentration Prediction Module:**
The extracted feature tensors from the meteorological module are concatenated with PM2.5 concentration data, forming the vertex attributes for a graph. Another GCN-LSTM structure processes this graph to predict PM2.5 concentrations

Pollution Data

Kaggle Dataset for beijing has only daily data, but starting from 2013 onwards
https://www.kaggle.com/datasets/sid321axn/beijing-multisite-airquality-data-set/data

import pandas as pd
pollution_data = pd.read_csv('pollution_data_export.csv',index_col=0)

## Current way to extract data: BY STATIONS AROUND LA BOUNDARY

In [131]:
#Checking what all countries are available

import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}

params = {
    'limit':10000
}

url = f'https://api.openaq.org/v2/countries'
response = requests.get(url, headers=headers, params=params)
data = response.json()
countries = (pd.DataFrame(data['results']))
countries

,code,name,locations,firstUpdated,lastUpdated,parameters,count,cities,sources
0,AF,Afghanistan,2,2019-10-20 23:00:00+00,2021-08-16 18:00:00+00,[pm25],23641,1,2
1,DZ,Algeria,1,2019-06-15 22:00:00+00,2024-10-22 21:00:00+00,[pm25],43012,1,1
2,AD,Andorra,2,2017-09-14 23:00:00+00,2024-10-21 14:00:00+00,"[co, no, no2, o3, pm10, pm25, so2]",297583,2,1
3,AQ,Antarctica,2,2018-11-30 11:00:00+00,2021-02-04 14:00:00+00,"[co, no2, o3, pm10, pm25, so2]",9180,0,2
4,AR,Argentina,18,2016-01-01 03:00:00+00,2024-10-23 03:00:00+00,"[co, no, no2, nox, o3, pm1, pm10, pm25, relati...",433042,1,6
...,...,...,...,...,...,...,...,...,...
126,US,United States,4316,2016-01-30 01:00:00+00,2024-10-22 22:07:05.956+00,"[bc, co, co2, no, no2, nox, o3, pm1, pm10, pm2...",327563408,788,9
127,UY,Uruguay,1,2023-10-19 22:00:00+00,2023-10-21 10:00:00+00,"[pm25, temperature]",66,0,1
128,UZ,Uzbekistan,1,2018-11-27 22:00:00+00,2024-10-22 21:00:00+00,"[o3, pm25]",84344,1,1
129,VN,Vietnam,40,2016-01-30 01:00:00+00,2024-10-22 21:00:00+00,"[co, no2, o3, pm10, pm25, so2, temperature]",406240,3,6


In [41]:
#Checking what all cities are available

import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}

params = {
    'limit':10000
}

url = f'https://api.openaq.org/v2/cities'
response = requests.get(url, headers=headers, params=params)
data = response.json()
cities = (pd.DataFrame(data['results']))
cities

,country,city,count,locations,firstUpdated,lastUpdated,parameters
0,DK,,23700,1,2017-11-09 23:00:00+00,2024-04-05 08:00:00+00,[no2]
1,JP,,58736481,1596,2023-07-14 17:00:00+00,2024-10-22 00:00:00+00,"[co, no, no2, nox, pm25, so2]"
2,KR,,19002641,713,2024-03-19 23:00:00+00,2024-10-22 01:00:00+00,"[co, no2, o3, pm10, pm25, so2]"
3,TT,,4231,1,2024-03-20 01:00:00+00,2024-07-08 13:00:00+00,"[co, pm10, pm25]"
4,IT,Brescia Via San Polo,5385,1,2024-01-28 23:00:00+00,2024-10-21 05:00:00+00,"[no2, pm25]"
...,...,...,...,...,...,...,...
4520,TW,花蓮縣,174478,1,2016-12-20 19:00:00+00,2024-10-22 01:00:00+00,"[co, no2, o3, pm10, pm25, so2]"
4521,TW,苗栗縣,99434,3,2024-02-26 21:00:00+00,2024-10-22 01:00:00+00,"[co, no2, o3, pm10, pm25, so2]"
4522,TW,金門縣,175388,1,2016-12-20 19:00:00+00,2024-10-22 01:00:00+00,"[co, no2, o3, pm10, pm25, so2]"
4523,TW,雲林縣,554982,4,2016-12-20 19:00:00+00,2024-10-22 01:00:00+00,"[co, no2, o3, pm10, pm25, so2]"


In [75]:
#Getting all stations within the boundary of LA Met Area
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':1000
}
url = f'https://api.openaq.org/v3/locations?bbox=-118.668153,33.703935,-118.155358,34.337306'
response = requests.get(url, headers=headers, params=params)
data = response.json()
LAdata=(pd.DataFrame(data['results']))
LAdata

,id,name,locality,timezone,country,owner,provider,isMobile,isMonitor,instruments,sensors,coordinates,licenses,bounds,distance,datetimeFirst,datetimeLast
0,847,South Long Beach,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 4, 'name': 'Unknown Governmental Organi...","{'id': 119, 'name': 'AirNow'}",False,True,"[{'id': 2, 'name': 'Government Monitor'}]","[{'id': 1502, 'name': 'pm25 µg/m³', 'parameter...","{'latitude': 33.792221, 'longitude': -118.175278}","[{'id': 33, 'name': 'US Public Domain', 'attri...","[-118.175278, 33.792221, -118.175278, 33.792221]",None,"{'utc': '2016-03-06T20:00:00Z', 'local': '2016...","{'utc': '2022-05-05T22:00:00Z', 'local': '2022..."
1,1247,LAX-Hastings,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 4, 'name': 'Unknown Governmental Organi...","{'id': 119, 'name': 'AirNow'}",False,True,"[{'id': 2, 'name': 'Government Monitor'}]","[{'id': 2244, 'name': 'o3 ppm', 'parameter': {...","{'latitude': 33.9517, 'longitude': -118.4389}","[{'id': 33, 'name': 'US Public Domain', 'attri...","[-118.4389, 33.9517, -118.4389, 33.9517]",None,"{'utc': '2016-03-06T20:00:00Z', 'local': '2016...","{'utc': '2016-11-09T21:00:00Z', 'local': '2016..."
2,1410,West Los Angeles - V,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 4, 'name': 'Unknown Governmental Organi...","{'id': 119, 'name': 'AirNow'}",False,True,"[{'id': 2, 'name': 'Government Monitor'}]","[{'id': 15732, 'name': 'co ppm', 'parameter': ...","{'latitude': 34.0506, 'longitude': -118.455300...","[{'id': 33, 'name': 'US Public Domain', 'attri...","[-118.45530000000001, 34.0506, -118.4553000000...",None,"{'utc': '2016-03-06T20:00:00Z', 'local': '2016...","{'utc': '2018-06-08T15:00:00Z', 'local': '2018..."
3,1564,Long Beach - Hudson,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 4, 'name': 'Unknown Governmental Organi...","{'id': 119, 'name': 'AirNow'}",False,True,"[{'id': 2, 'name': 'Government Monitor'}]","[{'id': 2754, 'name': 'o3 ppm', 'parameter': {...","{'latitude': 33.8022, 'longitude': -118.3667}","[{'id': 33, 'name': 'US Public Domain', 'attri...","[-118.3667, 33.8022, -118.3667, 33.8022]",None,"{'utc': '2016-03-06T20:00:00Z', 'local': '2016...","{'utc': '2016-11-09T21:00:00Z', 'local': '2016..."
4,1575,Los Angeles - N. Mai,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 4, 'name': 'Unknown Governmental Organi...","{'id': 119, 'name': 'AirNow'}",False,True,"[{'id': 2, 'name': 'Government Monitor'}]","[{'id': 8681, 'name': 'co ppm', 'parameter': {...","{'latitude': 34.0669, 'longitude': -118.2417}","[{'id': 33, 'name': 'US Public Domain', 'attri...","[-118.2417, 34.0669, -118.2417, 34.0669]",None,"{'utc': '2016-03-06T20:00:00Z', 'local': '2016...","{'utc': '2017-06-07T21:00:00Z', 'local': '2017..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2861424,12049 Stanley Park Ct.,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 12, 'name': 'AirGradient'}","{'id': 66, 'name': 'AirGradient'}",False,False,"[{'id': 7, 'name': 'AirGradient Sensor'}]","[{'id': 9329456, 'name': 'pm1 µg/m³', 'paramet...","{'latitude': 33.921251, 'longitude': -118.319129}","[{'id': 41, 'name': 'CC BY 4.0 DEED', 'attribu...","[-118.319129, 33.921251, -118.319129, 33.921251]",None,"{'utc': '2024-05-23T03:00:00Z', 'local': '2024...","{'utc': '2024-10-22T20:00:00Z', 'local': '2024..."
282,2945078,N. Main Street Collocation,None,America/Los_Angeles,"{'id': 155, 'code': 'US', 'name': 'United Stat...","{'id': 9, 'name': 'Clarity'}","{'id': 166, 'name': 'Clarity'}",False,False,"[{'id': 4, 'name': 'Clarity Sensor'}]","[{'id': 10141094, 'name': 'pm25 µg/m³', 'param...","{'latitude': 34.06639, 'longitude': -118.22667}","[{'id': 38, 'name': 'CC0 1.0 DEED', 'attributi...","[-118.22667, 34.06639, -118.22667, 34.06639]",None,"{'utc': '2024-07-29T00:00:00Z

Then I manually filtered the list to stations that have data within the range of 2022-01 to 2022-12

Now querying data station-wise

In [112]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290486,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s1 = pd.DataFrame(data['resu lts'])
s1

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290486,8544 - Roybal Lc,pm25,6.78,"{'utc': '2022-03-11T16:31:39+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
1,290486,8544 - Roybal Lc,pm25,6.32,"{'utc': '2022-03-11T16:18:36+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
2,290486,8544 - Roybal Lc,pm25,6.06,"{'utc': '2022-03-11T16:12:24+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
3,290486,8544 - Roybal Lc,pm25,6.35,"{'utc': '2022-03-11T16:06:17+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
4,290486,8544 - Roybal Lc,pm25,6.35,"{'utc': '2022-03-11T15:39:16+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,290486,8544 - Roybal Lc,pm25,5.54,"{'utc': '2022-02-18T02:20:42+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
4537,290486,8544 - Roybal Lc,pm25,5.59,"{'utc': '2022-02-18T02:14:33+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
4538,290486,8544 - Roybal Lc,pm25,6.23,"{'utc': '2022-02-18T02:08:24+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
4539,290486,8544 - Roybal Lc,pm25,5.32,"{'utc': '2022-02-18T02:02:12+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor


In [111]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290493,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s2 = pd.DataFrame(data['results'])
s2

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290493,5857 - 107Th St El,pm25,35.78,"{'utc': '2022-03-03T16:39:14+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
1,290493,5857 - 107Th St El,pm25,37.21,"{'utc': '2022-03-03T16:33:06+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2,290493,5857 - 107Th St El,pm25,37.83,"{'utc': '2022-03-03T16:27:01+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
3,290493,5857 - 107Th St El,pm25,36.77,"{'utc': '2022-03-03T16:20:46+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
4,290493,5857 - 107Th St El,pm25,35.32,"{'utc': '2022-03-03T16:14:35+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,290493,5857 - 107Th St El,pm25,5.74,"{'utc': '2022-02-18T02:20:25+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2875,290493,5857 - 107Th St El,pm25,6.02,"{'utc': '2022-02-18T02:14:14+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2876,290493,5857 - 107Th St El,pm25,5.85,"{'utc': '2022-02-18T02:08:03+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2877,290493,5857 - 107Th St El,pm25,5.50,"{'utc': '2022-02-18T02:01:49+00:00', 'local': ...",µg/m³,"{'latitude': 33.94018143, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor


In [110]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290494,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s3 = pd.DataFrame(data['results'])
s3

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290494,2945 - Century Pk El,pm25,4.99,"{'utc': '2022-03-03T17:13:45+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
1,290494,2945 - Century Pk El,pm25,6.17,"{'utc': '2022-03-03T17:08:01+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
2,290494,2945 - Century Pk El,pm25,6.59,"{'utc': '2022-03-03T17:02:17+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
3,290494,2945 - Century Pk El,pm25,6.19,"{'utc': '2022-03-03T16:56:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
4,290494,2945 - Century Pk El,pm25,3.38,"{'utc': '2022-03-03T16:50:48+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,290494,2945 - Century Pk El,pm25,5.38,"{'utc': '2022-02-18T02:17:49+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
3131,290494,2945 - Century Pk El,pm25,5.33,"{'utc': '2022-02-18T02:12:04+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
3132,290494,2945 - Century Pk El,pm25,5.81,"{'utc': '2022-02-18T02:06:19+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor
3133,290494,2945 - Century Pk El,pm25,5.71,"{'utc': '2022-02-18T02:00:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.93643353, 'longitude': -118.31...",US,None,False,None,Community Organization,low-cost sensor


In [97]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290497
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s4 = pd.DataFrame(data['results'])
s4

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290497,3548 - Elizabeth Learning Ctr,pm25,16.50,"{'utc': '2022-03-03T21:18:38+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
1,290497,3548 - Elizabeth Learning Ctr,pm25,12.40,"{'utc': '2022-03-03T21:12:40+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
2,290497,3548 - Elizabeth Learning Ctr,pm25,9.67,"{'utc': '2022-03-03T21:06:42+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
3,290497,3548 - Elizabeth Learning Ctr,pm25,11.86,"{'utc': '2022-03-03T21:00:41+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
4,290497,3548 - Elizabeth Learning Ctr,pm25,12.55,"{'utc': '2022-03-03T20:54:40+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8896,290497,3548 - Elizabeth Learning Ctr,pm10,4.42,"{'utc': '2022-02-18T02:18:00+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
8897,290497,3548 - Elizabeth Learning Ctr,pm10,5.19,"{'utc': '2022-02-18T02:11:56+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
8898,290497,3548 - Elizabeth Learning Ctr,pm10,6.62,"{'utc': '2022-02-18T02:05:51+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
8899,290497,3548 - Elizabeth Learning Ctr,pm10,6.47,"{'utc': '2022-02-18T01:59:44+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor


In [109]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290497,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s5 = pd.DataFrame(data['results'])
s5

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290497,3548 - Elizabeth Learning Ctr,pm25,16.50,"{'utc': '2022-03-03T21:18:38+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
1,290497,3548 - Elizabeth Learning Ctr,pm25,12.40,"{'utc': '2022-03-03T21:12:40+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
2,290497,3548 - Elizabeth Learning Ctr,pm25,9.67,"{'utc': '2022-03-03T21:06:42+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
3,290497,3548 - Elizabeth Learning Ctr,pm25,11.86,"{'utc': '2022-03-03T21:00:41+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
4,290497,3548 - Elizabeth Learning Ctr,pm25,12.55,"{'utc': '2022-03-03T20:54:40+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,290497,3548 - Elizabeth Learning Ctr,pm25,5.46,"{'utc': '2022-02-18T02:18:00+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
2963,290497,3548 - Elizabeth Learning Ctr,pm25,5.78,"{'utc': '2022-02-18T02:11:56+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
2964,290497,3548 - Elizabeth Learning Ctr,pm25,6.09,"{'utc': '2022-02-18T02:05:51+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor
2965,290497,3548 - Elizabeth Learning Ctr,pm25,6.20,"{'utc': '2022-02-18T01:59:44+00:00', 'local': ...",µg/m³,"{'latitude': 33.96405384, 'longitude': -118.18...",US,None,False,None,Community Organization,low-cost sensor


In [108]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290527,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s6 = pd.DataFrame(data['results'])
s6

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290527,Gulf ES (4315),pm25,6.67,"{'utc': '2022-06-01T19:04:37+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
1,290527,Gulf ES (4315),pm25,5.84,"{'utc': '2022-06-01T18:58:57+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2,290527,Gulf ES (4315),pm25,6.43,"{'utc': '2022-06-01T18:53:17+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
3,290527,Gulf ES (4315),pm25,6.09,"{'utc': '2022-06-01T18:47:37+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
4,290527,Gulf ES (4315),pm25,6.18,"{'utc': '2022-06-01T18:41:49+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17057,290527,Gulf ES (4315),pm25,5.68,"{'utc': '2022-03-03T17:48:44+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
17058,290527,Gulf ES (4315),pm25,5.82,"{'utc': '2022-03-03T17:43:04+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
17059,290527,Gulf ES (4315),pm25,5.61,"{'utc': '2022-03-03T17:37:24+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
17060,290527,Gulf ES (4315),pm25,5.59,"{'utc': '2022-03-03T17:31:43+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor


In [105]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290539,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s7 = pd.DataFrame(data['results'])
s7

KeyError: 'results'

In [106]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290548,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s8 = pd.DataFrame(data['results'])
s8

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290548,4521 - Hesby Oaks,pm25,11.63,"{'utc': '2022-04-08T15:35:05+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
1,290548,4521 - Hesby Oaks,pm25,6.11,"{'utc': '2022-04-08T15:29:26+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
2,290548,4521 - Hesby Oaks,pm25,5.59,"{'utc': '2022-04-08T15:23:51+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
3,290548,4521 - Hesby Oaks,pm25,12.23,"{'utc': '2022-04-08T15:18:16+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
4,290548,4521 - Hesby Oaks,pm25,6.36,"{'utc': '2022-04-08T15:12:40+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11678,290548,4521 - Hesby Oaks,pm25,4.97,"{'utc': '2022-02-18T02:16:59+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11679,290548,4521 - Hesby Oaks,pm25,4.65,"{'utc': '2022-02-18T02:11:23+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11680,290548,4521 - Hesby Oaks,pm25,4.85,"{'utc': '2022-02-18T02:05:49+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11681,290548,4521 - Hesby Oaks,pm25,4.80,"{'utc': '2022-02-18T02:00:13+00:00', 'local': ...",µg/m³,"{'latitude': 34.16132205, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor


In [107]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290556,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s9 = pd.DataFrame(data['results'])
s9

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290556,3959 - 42Nd St El,pm25,4.62,"{'utc': '2022-03-11T16:46:41+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
1,290556,3959 - 42Nd St El,pm25,4.70,"{'utc': '2022-03-11T16:33:54+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
2,290556,3959 - 42Nd St El,pm25,5.86,"{'utc': '2022-03-11T16:27:57+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
3,290556,3959 - 42Nd St El,pm25,3.39,"{'utc': '2022-03-11T16:22:00+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
4,290556,3959 - 42Nd St El,pm25,4.33,"{'utc': '2022-03-11T16:09:16+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,290556,3959 - 42Nd St El,pm25,4.61,"{'utc': '2022-02-18T02:21:55+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
4739,290556,3959 - 42Nd St El,pm25,5.44,"{'utc': '2022-02-18T02:15:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
4740,290556,3959 - 42Nd St El,pm25,5.12,"{'utc': '2022-02-18T02:09:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor
4741,290556,3959 - 42Nd St El,pm25,4.56,"{'utc': '2022-02-18T02:03:51+00:00', 'local': ...",µg/m³,"{'latitude': 34.0063417, 'longitude': -118.322...",US,None,False,None,Community Organization,low-cost sensor


In [115]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290562,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s10 = pd.DataFrame(data['results'])
s10

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290562,7781 - Wilmington Pk El,pm25,8.18,"{'utc': '2022-03-03T18:38:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
1,290562,7781 - Wilmington Pk El,pm25,7.80,"{'utc': '2022-03-03T18:32:26+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
2,290562,7781 - Wilmington Pk El,pm25,10.07,"{'utc': '2022-03-03T18:26:23+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
3,290562,7781 - Wilmington Pk El,pm25,9.57,"{'utc': '2022-03-03T18:20:16+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
4,290562,7781 - Wilmington Pk El,pm25,9.35,"{'utc': '2022-03-03T18:14:12+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2959,290562,7781 - Wilmington Pk El,pm25,4.51,"{'utc': '2022-02-18T02:20:47+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
2960,290562,7781 - Wilmington Pk El,pm25,4.84,"{'utc': '2022-02-18T02:14:44+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
2961,290562,7781 - Wilmington Pk El,pm25,4.96,"{'utc': '2022-02-18T02:08:40+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor
2962,290562,7781 - Wilmington Pk El,pm25,5.25,"{'utc': '2022-02-18T02:02:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.78616669, 'longitude': -118.24...",US,None,False,None,Community Organization,low-cost sensor


In [116]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290586,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s11 = pd.DataFrame(data['results'])
s11

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290586,5986 - Palms El,pm25,0.00,"{'utc': '2022-10-21T17:06:32+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
1,290586,5986 - Palms El,pm25,0.00,"{'utc': '2022-10-21T17:00:55+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
2,290586,5986 - Palms El,pm25,0.00,"{'utc': '2022-10-21T16:55:18+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
3,290586,5986 - Palms El,pm25,0.00,"{'utc': '2022-10-21T16:49:42+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
4,290586,5986 - Palms El,pm25,106.82,"{'utc': '2022-10-21T16:44:05+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57750,290586,5986 - Palms El,pm25,5.18,"{'utc': '2022-02-18T02:17:07+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
57751,290586,5986 - Palms El,pm25,5.03,"{'utc': '2022-02-18T02:11:31+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
57752,290586,5986 - Palms El,pm25,4.96,"{'utc': '2022-02-18T02:05:54+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor
57753,290586,5986 - Palms El,pm25,5.66,"{'utc': '2022-02-18T02:00:18+00:00', 'local': ...",µg/m³,"{'latitude': 34.0260643, 'longitude': -118.40747}",US,None,False,None,Community Organization,low-cost sensor


In [117]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290592,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s12 = pd.DataFrame(data['results'])
s12

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290592,6137 - Point Fermin Elementary,pm25,5.23,"{'utc': '2022-03-03T19:25:52+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
1,290592,6137 - Point Fermin Elementary,pm25,4.99,"{'utc': '2022-03-03T19:11:44+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
2,290592,6137 - Point Fermin Elementary,pm25,5.15,"{'utc': '2022-03-03T18:57:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
3,290592,6137 - Point Fermin Elementary,pm25,5.53,"{'utc': '2022-03-03T18:43:25+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
4,290592,6137 - Point Fermin Elementary,pm25,5.83,"{'utc': '2022-03-03T18:29:17+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,290592,6137 - Point Fermin Elementary,pm25,3.77,"{'utc': '2022-02-18T02:14:39+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
2689,290592,6137 - Point Fermin Elementary,pm25,4.33,"{'utc': '2022-02-18T02:09:09+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
2690,290592,6137 - Point Fermin Elementary,pm25,4.20,"{'utc': '2022-02-18T02:03:33+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor
2691,290592,6137 - Point Fermin Elementary,pm25,4.23,"{'utc': '2022-02-18T01:58:03+00:00', 'local': ...",µg/m³,"{'latitude': 33.7142556, 'longitude': -118.290...",US,None,False,None,Community Organization,low-cost sensor


In [118]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290598,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s13 = pd.DataFrame(data['results'])
s13

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290598,4315 - Gulf El,pm25,5.20,"{'utc': '2022-03-01T15:36:11+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
1,290598,4315 - Gulf El,pm25,5.93,"{'utc': '2022-02-28T23:42:07+00:00', 'local': ...",µg/m³,"{'latitude': 33.78548487, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor


In [119]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290612,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s14 = pd.DataFrame(data['results'])
s14

KeyError: 'results'

In [120]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290613,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s15 = pd.DataFrame(data['results'])
s15

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290613,Sheridan St ES (6685),pm25,12.57,"{'utc': '2022-04-10T01:25:29+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
1,290613,Sheridan St ES (6685),pm25,11.74,"{'utc': '2022-04-10T01:19:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
2,290613,Sheridan St ES (6685),pm25,12.56,"{'utc': '2022-04-10T01:14:14+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
3,290613,Sheridan St ES (6685),pm25,11.29,"{'utc': '2022-04-10T01:08:37+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
4,290613,Sheridan St ES (6685),pm25,11.72,"{'utc': '2022-04-10T01:02:59+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10782,290613,Sheridan St ES (6685),pm25,0.19,"{'utc': '2022-02-18T02:15:07+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
10783,290613,Sheridan St ES (6685),pm25,0.38,"{'utc': '2022-02-18T02:09:29+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
10784,290613,Sheridan St ES (6685),pm25,0.00,"{'utc': '2022-02-18T02:03:51+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor
10785,290613,Sheridan St ES (6685),pm25,0.00,"{'utc': '2022-02-18T01:58:13+00:00', 'local': ...",µg/m³,"{'latitude': 34.0494627, 'longitude': -118.207...",US,None,False,None,Community Organization,low-cost sensor


In [121]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290617,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s16 = pd.DataFrame(data['results'])
s16

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290617,2383 - Esperanza El,pm25,10.68,"{'utc': '2022-07-24T03:19:22+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
1,290617,2383 - Esperanza El,pm25,12.53,"{'utc': '2022-07-24T03:13:46+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
2,290617,2383 - Esperanza El,pm25,10.98,"{'utc': '2022-07-24T03:08:10+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
3,290617,2383 - Esperanza El,pm25,11.34,"{'utc': '2022-07-24T03:02:34+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
4,290617,2383 - Esperanza El,pm25,11.24,"{'utc': '2022-07-24T02:56:58+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36949,290617,2383 - Esperanza El,pm25,6.60,"{'utc': '2022-02-18T02:16:22+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
36950,290617,2383 - Esperanza El,pm25,6.36,"{'utc': '2022-02-18T02:10:46+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
36951,290617,2383 - Esperanza El,pm25,5.41,"{'utc': '2022-02-18T02:05:09+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor
36952,290617,2383 - Esperanza El,pm25,4.98,"{'utc': '2022-02-18T01:59:33+00:00', 'local': ...",µg/m³,"{'latitude': 34.05495895, 'longitude': -118.27...",US,None,False,None,Community Organization,low-cost sensor


In [122]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290619,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s17 = pd.DataFrame(data['results'])
s17

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290619,4790 - Lassen Es,pm25,5.82,"{'utc': '2022-03-22T14:21:28+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
1,290619,4790 - Lassen Es,pm25,5.75,"{'utc': '2022-03-22T14:15:56+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
2,290619,4790 - Lassen Es,pm25,5.45,"{'utc': '2022-03-22T14:10:22+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
3,290619,4790 - Lassen Es,pm25,5.49,"{'utc': '2022-03-22T14:04:50+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
4,290619,4790 - Lassen Es,pm25,5.78,"{'utc': '2022-03-22T13:59:17+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7691,290619,4790 - Lassen Es,pm25,6.42,"{'utc': '2022-02-18T02:19:50+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
7692,290619,4790 - Lassen Es,pm25,6.20,"{'utc': '2022-02-18T02:14:17+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
7693,290619,4790 - Lassen Es,pm25,6.84,"{'utc': '2022-02-18T02:08:44+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor
7694,290619,4790 - Lassen Es,pm25,6.14,"{'utc': '2022-02-18T02:03:11+00:00', 'local': ...",µg/m³,"{'latitude': 34.24714121, 'longitude': -118.45...",US,None,False,None,Community Organization,low-cost sensor


In [123]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290627,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s18 = pd.DataFrame(data['results'])
s18

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290627,4096 - Gates El,pm25,7.34,"{'utc': '2022-03-14T04:51:32+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
1,290627,4096 - Gates El,pm25,9.33,"{'utc': '2022-03-14T04:45:51+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
2,290627,4096 - Gates El,pm25,5.41,"{'utc': '2022-03-14T04:40:10+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
3,290627,4096 - Gates El,pm25,5.42,"{'utc': '2022-03-14T04:34:29+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
4,290627,4096 - Gates El,pm25,7.94,"{'utc': '2022-03-14T04:28:47+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697,290627,4096 - Gates El,pm25,5.02,"{'utc': '2022-02-18T02:20:13+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5698,290627,4096 - Gates El,pm25,5.34,"{'utc': '2022-02-18T02:14:32+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5699,290627,4096 - Gates El,pm25,5.66,"{'utc': '2022-02-18T02:08:50+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5700,290627,4096 - Gates El,pm25,5.10,"{'utc': '2022-02-18T02:03:09+00:00', 'local': ...",µg/m³,"{'latitude': 34.07282985, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor


In [124]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290631,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s19 = pd.DataFrame(data['results'])
s19

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290631,2603 - Buchanan El,pm25,4.09,"{'utc': '2022-03-20T17:20:41+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
1,290631,2603 - Buchanan El,pm25,4.20,"{'utc': '2022-03-20T16:51:46+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
2,290631,2603 - Buchanan El,pm25,4.47,"{'utc': '2022-03-20T16:39:59+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
3,290631,2603 - Buchanan El,pm25,9.14,"{'utc': '2022-03-19T23:35:06+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
4,290631,2603 - Buchanan El,pm25,8.46,"{'utc': '2022-03-19T23:23:19+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5582,290631,2603 - Buchanan El,pm25,5.38,"{'utc': '2022-02-18T02:19:35+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5583,290631,2603 - Buchanan El,pm25,5.56,"{'utc': '2022-02-18T02:14:01+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5584,290631,2603 - Buchanan El,pm25,5.27,"{'utc': '2022-02-18T02:08:26+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor
5585,290631,2603 - Buchanan El,pm25,4.91,"{'utc': '2022-02-18T02:02:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.11908877, 'longitude': -118.20...",US,None,False,None,Community Organization,low-cost sensor


In [125]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290647,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s20 = pd.DataFrame(data['results'])
s20

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290647,5342 - Monlux El,pm25,0.00,"{'utc': '2022-05-19T18:44:09+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
1,290647,5342 - Monlux El,pm25,0.00,"{'utc': '2022-05-19T18:38:26+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
2,290647,5342 - Monlux El,pm25,0.00,"{'utc': '2022-05-19T18:32:43+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
3,290647,5342 - Monlux El,pm25,0.00,"{'utc': '2022-05-19T18:27:00+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
4,290647,5342 - Monlux El,pm25,0.00,"{'utc': '2022-05-19T18:21:17+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19372,290647,5342 - Monlux El,pm25,5.85,"{'utc': '2022-02-18T02:19:21+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
19373,290647,5342 - Monlux El,pm25,5.72,"{'utc': '2022-02-18T02:13:35+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
19374,290647,5342 - Monlux El,pm25,5.63,"{'utc': '2022-02-18T02:07:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor
19375,290647,5342 - Monlux El,pm25,5.15,"{'utc': '2022-02-18T02:02:10+00:00', 'local': ...",µg/m³,"{'latitude': 34.18168176, 'longitude': -118.40...",US,None,False,None,Community Organization,low-cost sensor


In [126]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':290676,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s21 = pd.DataFrame(data['results'])
s21

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290676,7562 - Vintage Math/Sci Mag,pm25,4.82,"{'utc': '2022-04-08T15:28:54+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
1,290676,7562 - Vintage Math/Sci Mag,pm25,5.04,"{'utc': '2022-04-08T15:23:22+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
2,290676,7562 - Vintage Math/Sci Mag,pm25,5.11,"{'utc': '2022-04-08T15:17:50+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
3,290676,7562 - Vintage Math/Sci Mag,pm25,5.80,"{'utc': '2022-04-08T15:12:19+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
4,290676,7562 - Vintage Math/Sci Mag,pm25,6.74,"{'utc': '2022-04-08T15:06:47+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11830,290676,7562 - Vintage Math/Sci Mag,pm25,4.73,"{'utc': '2022-02-18T02:15:41+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11831,290676,7562 - Vintage Math/Sci Mag,pm25,4.98,"{'utc': '2022-02-18T02:10:10+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11832,290676,7562 - Vintage Math/Sci Mag,pm25,4.80,"{'utc': '2022-02-18T02:04:38+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor
11833,290676,7562 - Vintage Math/Sci Mag,pm25,6.41,"{'utc': '2022-02-18T01:59:07+00:00', 'local': ...",µg/m³,"{'latitude': 34.25188797, 'longitude': -118.47...",US,None,False,None,Community Organization,low-cost sensor


In [127]:
import requests
api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
headers = {'x-api-key': api_key}
params = {
    'date_from': '2022-01-01',
    'date_to': '2022-12-31',
    'format': 'json',
    'limit':100000,'location_id':351827,
    'parameter':'pm25'
}
url = f'https://api.openaq.org/v2/measurements'
response = requests.get(url, headers=headers, params=params)
data = response.json()
s22 = pd.DataFrame(data['results'])
s22

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,351827,AXJXGCPF,pm25,12.58,"{'utc': '2022-06-22T21:13:26+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
1,351827,AXJXGCPF,pm25,12.65,"{'utc': '2022-06-22T20:56:15+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
2,351827,AXJXGCPF,pm25,10.42,"{'utc': '2022-06-22T20:39:05+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
3,351827,AXJXGCPF,pm25,6.92,"{'utc': '2022-06-22T20:21:52+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
4,351827,AXJXGCPF,pm25,7.35,"{'utc': '2022-06-22T20:04:41+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,351827,AXJXGCPF,pm25,44.45,"{'utc': '2022-06-10T19:36:34+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
963,351827,AXJXGCPF,pm25,45.20,"{'utc': '2022-06-10T19:19:24+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
964,351827,AXJXGCPF,pm25,45.76,"{'utc': '2022-06-10T19:02:15+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
965,351827,AXJXGCPF,pm25,46.22,"{'utc': '2022-06-10T18:44:56+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor


In [129]:
final_data = pd.concat([s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s15,s16,s17,s18,s19,s20,s21,s22], ignore_index=True)
final_data

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,290486,8544 - Roybal Lc,pm25,6.78,"{'utc': '2022-03-11T16:31:39+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
1,290486,8544 - Roybal Lc,pm25,6.32,"{'utc': '2022-03-11T16:18:36+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
2,290486,8544 - Roybal Lc,pm25,6.06,"{'utc': '2022-03-11T16:12:24+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
3,290486,8544 - Roybal Lc,pm25,6.35,"{'utc': '2022-03-11T16:06:17+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
4,290486,8544 - Roybal Lc,pm25,6.35,"{'utc': '2022-03-11T15:39:16+00:00', 'local': ...",µg/m³,"{'latitude': 34.06177624, 'longitude': -118.25...",US,None,False,None,Community Organization,low-cost sensor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253274,351827,AXJXGCPF,pm25,44.45,"{'utc': '2022-06-10T19:36:34+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
253275,351827,AXJXGCPF,pm25,45.20,"{'utc': '2022-06-10T19:19:24+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
253276,351827,AXJXGCPF,pm25,45.76,"{'utc': '2022-06-10T19:02:15+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor
253277,351827,AXJXGCPF,pm25,46.22,"{'utc': '2022-06-10T18:44:56+00:00', 'local': ...",µg/m³,"{'latitude': 34.19444, 'longitude': -118.157795}",US,None,False,None,Community Organization,low-cost sensor


In [130]:
final_data.to_csv('LAPollutionData.csv')

## Alternate way to extract data: BY DATE (Doesn't work)

In [26]:
import requests
import pandas as pd

def get_pm25_data(date_from,date_to):
    # API key and headers
    api_key = '6701a271856184c8160a38297861c796ac408e69ca7be88c9f358a122bc2378a'
    headers = {'x-api-key': api_key}

    params = {
        'city': 'Los Angeles',
        'parameter': 'pm25',
        'date_from': date_from,
        'date_to': date_to,
        'format': 'json',
        'limit':10000
    }

    url = f'https://api.openaq.org/v2/measurements'
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    print(data)

    pm25_data = []
    for record in data['results']:
      pm25_data.append({
              'location': record['location'],
              'city': record['city'],
              'country': record['country'],
              'latitude': record['coordinates']['latitude'],
              'longitude': record['coordinates']['longitude'],
              'pm25': record['value'],
              'unit': record['unit'],
              'date': record['date']['utc']
          })

    df = pd.DataFrame(pm25_data)
    return df

# # pm25_data = []
# for i in month_start_dates:
#   for j in month_end_dates:
#     pm25_data.append(get_pm25_data(i,j))

# print(pm25_data.head())

In [27]:
pm25_jan = get_pm25_data('2022-01-01T00:00:00+00:00','2022-01-31T00:00:00+00:00')
pm25_jan

{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 10000, 'found': 1364}, 'results': [{'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 13.0, 'date': {'utc': '2022-01-31T00:00:00+00:00', 'local': '2022-01-30T21:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 13.0, 'date': {'utc': '2022-01-30T23:00:00+00:00', 'local': '2022-01-30T20:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', '

,location,city,country,latitude,longitude,pm25,unit,date
0,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-01-31T00:00:00+00:00
1,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-01-30T23:00:00+00:00
2,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-01-30T22:00:00+00:00
3,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-01-30T21:00:00+00:00
4,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,14.0,µg/m³,2022-01-30T20:00:00+00:00
...,...,...,...,...,...,...,...,...
1359,21 de mayo,None,CL,-37.471182,-72.361463,17.0,µg/m³,2022-01-01T05:00:00+00:00
1360,21 de mayo,None,CL,-37.471182,-72.361463,17.0,µg/m³,2022-01-01T04:00:00+00:00
1361,21 de mayo,None,CL,-37.471182,-72.361463,17.0,µg/m³,2022-01-01T03:00:00+00:00
1362,21 de mayo,None,CL,-37.471182,-72.361463,17.0,µg/m³,2022-01-01T02:00:00+00:00


In [28]:
pm25_feb = get_pm25_data('2022-02-01T00:00:00+00:00','2022-02-28T00:00:00+00:00')
pm25_feb

{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 10000, 'found': 1214}, 'results': [{'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 13.0, 'date': {'utc': '2022-02-28T00:00:00+00:00', 'local': '2022-02-27T21:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 13.0, 'date': {'utc': '2022-02-27T23:00:00+00:00', 'local': '2022-02-27T20:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', '

,location,city,country,latitude,longitude,pm25,unit,date
0,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-02-28T00:00:00+00:00
1,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-02-27T23:00:00+00:00
2,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-02-27T22:00:00+00:00
3,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-02-27T21:00:00+00:00
4,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,13.0,µg/m³,2022-02-27T20:00:00+00:00
...,...,...,...,...,...,...,...,...
1209,21 de mayo,None,CL,-37.471182,-72.361463,18.0,µg/m³,2022-02-01T05:00:00+00:00
1210,21 de mayo,None,CL,-37.471182,-72.361463,18.0,µg/m³,2022-02-01T04:00:00+00:00
1211,21 de mayo,None,CL,-37.471182,-72.361463,19.0,µg/m³,2022-02-01T03:00:00+00:00
1212,21 de mayo,None,CL,-37.471182,-72.361463,19.0,µg/m³,2022-02-01T02:00:00+00:00


In [29]:
pm25_mar = get_pm25_data('2022-03-01T00:00:00+00:00','2022-03-31T00:00:00+00:00')
pm25_mar

{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 10000, 'found': 1328}, 'results': [{'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 18.0, 'date': {'utc': '2022-03-31T00:00:00+00:00', 'local': '2022-03-30T21:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', 'parameter': 'pm25', 'value': 18.0, 'date': {'utc': '2022-03-30T23:00:00+00:00', 'local': '2022-03-30T20:00:00-03:00'}, 'unit': 'µg/m³', 'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457}, 'country': 'CL', 'city': None, 'isMobile': False, 'isAnalysis': None, 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}, {'locationId': 270, 'location': 'Los Ángeles Oriente', '

,location,city,country,latitude,longitude,pm25,unit,date
0,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,18.0,µg/m³,2022-03-31T00:00:00+00:00
1,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,18.0,µg/m³,2022-03-30T23:00:00+00:00
2,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,18.0,µg/m³,2022-03-30T22:00:00+00:00
3,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,17.0,µg/m³,2022-03-30T21:00:00+00:00
4,Los Ángeles Oriente,None,CL,-37.463035,-72.324575,16.0,µg/m³,2022-03-30T20:00:00+00:00
...,...,...,...,...,...,...,...,...
1323,21 de mayo,None,CL,-37.471182,-72.361463,19.0,µg/m³,2022-03-01T05:00:00+00:00
1324,21 de mayo,None,CL,-37.471182,-72.361463,19.0,µg/m³,2022-03-01T04:00:00+00:00
1325,21 de mayo,None,CL,-37.471182,-72.361463,18.0,µg/m³,2022-03-01T03:00:00+00:00
1326,21 de mayo,None,CL,-37.471182,-72.361463,18.0,µg/m³,2022-03-01T02:00:00+00:00


# METEOROLOGICAL DATA (WIP)

In [93]:
import requests
import pandas as pd

# NOAA API Key (replace with your actual API key)
API_KEY = "BAkKPJfPHPITQtzJhckCttPQnfvCzuFP"

# Base URL for the NOAA API
BASE_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"

# Function to make a request to the NOAA API
def get_noaa_data(dataset_id, location_id, start_date, end_date, limit=1000, offset=0):
    headers = {
        "token": API_KEY
    }
    params = {
        "datasetid": dataset_id,
        "locationid": location_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": limit,
        "offset": offset
    }
    response = requests.get(BASE_URL + "data", headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Function to handle pagination and collect all data
def fetch_all_data(dataset_id, location_id, start_date, end_date):
    limit = 1000  # Maximum records per request
    offset = 0
    all_data = []

    while True:
        data = get_noaa_data(dataset_id, location_id, start_date, end_date, limit, offset)
        if data and "results" in data:
            all_data.extend(data["results"])
            if len(data["results"]) < limit:
                break  # No more data to fetch
            offset += limit
        else:
            break

    return all_data

# Example usage: Fetch data from GHCND dataset for a specific location
dataset_id = "GHCND"  # Global Historical Climatology Network Daily
location_id = "FIPS:06"  # Example: FIPS code for North Carolina
start_date = "2022-01-01"
end_date = "2022-12-31"

# Fetch all the data
data = fetch_all_data(dataset_id, location_id, start_date, end_date)

# Convert the data into a DataFrame for easier handling
if data:
    df = pd.DataFrame(data)
    print(df.head())
else:
    print("No data returned.")

# Optional: Save to CSV
df


Error: 502
No data returned.


NameError: name 'df' is not defined